In [1]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import pandas as pd
import copy
import random
from torch.utils.data import Dataset, DataLoader, BatchSampler, RandomSampler, WeightedRandomSampler
from torch.utils.data.sampler import Sampler
from torch import linalg as la
from sklearn import preprocessing


class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        self.l1 = nn.Linear(input_dim, output_dim)
    def forward(self, x):
        x = self.l1(x)
        return x  
    

#Reads Onr dataset
class CSVDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.features = dataset.iloc[:,:-1].values
        self.labels = dataset.iloc[:,-1].values
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        feature = self.features[idx]
        feature = torch.tensor(feature, dtype = torch.float)
        label = self.labels[idx]
        label = torch.tensor(label, dtype = torch.float)
        return feature, label
    

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    def __len__(self):
        return len(self.dl)


def zero_grad(params):
    for p in params:
        if p.grad is not None:
            p.grad.detach()
            p.grad.zero_()

            
def total_loss(model, loss_fns, dataloader, n):
    total_loss = 0
    for x, y in dataloader:
        out = model(x)
        loss = loss_fns(out, y.float().view(x.shape[0], -1))
        total_loss = total_loss + loss.item()
    return total_loss * (1/n)


def test_loss(model, loss_fns, test_dataloader, n):
    test_loss = 0
    for x, y in test_dataloader:
        out = model(x)
        loss = loss_fns(out, y.float().view(x.shape[0], -1))
        test_loss = test_loss + loss.item()
    return test_loss * (1/n)
 
    
def total_grad(model, loss_fns, dataloader, n):
    total_grad = 0
    zero_grad(list(model.parameters()))
    for x, y in dataloader:
        out = model(x)
        loss = loss_fns(out, y.float().view(x.shape[0], -1))
        loss.backward()
    for p in list(model.parameters()):
        total_grad = total_grad + torch.sum(torch.square(torch.mul(torch.clone(p.grad.data).detach(), (1/n))))
    zero_grad(list(model.parameters()))
    return torch.sqrt(torch.clone(total_grad).detach())


def full_grad(model, loss_fns, dataloader, n):
    full_grad = []
    zero_grad(list(model.parameters()))
    for x, y in dataloader:
        out = model(x)
        loss = loss_fns(out, y.float().view(x.shape[0], -1))
        loss.backward()
    for p in list(model.parameters()):
        full_grad.append(torch.mul(torch.clone(p.grad.data).detach(), (1/n)))
    zero_grad(list(model.parameters()))
    return full_grad 


def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)


#Reads Onr dataset, scales and transforms it into class of 'torch.dataset'
onr_data = pd.read_csv('OnlineNewsPopularity.csv', dtype = np.float32)
MaxminScaler = preprocessing.MinMaxScaler()
onr_data = MaxminScaler.fit_transform(onr_data)
onr_data = pd.DataFrame(onr_data)
onr_dataset = CSVDataset(onr_data)     
d = 58
nt = 39644
n = 31715
n1 = nt - n
epoches = 100
weight_decay = 1e-6
rbs = 500
#Loss function that is used to record the training, testing losses and l2-norm of full gradients
loss_func_rec = nn.MSELoss(reduction='sum')
#Split the Onr dataset into training set and testing set.
generator1 = torch.Generator().manual_seed(42)
onr_train, onr_test = torch.utils.data.random_split(onr_dataset, [n,n1], generator = generator1)
#Dataloaders which are used to calculate the training, testing losses and l2-norm of full gradients
onr_train_loader = DataLoader(onr_train, batch_size = rbs)
onr_train_loader = DeviceDataLoader(onr_train_loader, device)
onr_test_loader = DataLoader(onr_test, batch_size = rbs)
onr_test_loader = DeviceDataLoader(onr_test_loader, device)

In [3]:
#training stage: Linear regression with l^2 regularization using ggd-adam 
#stage one: preparation, initialization and hyperparameter setting
device = torch.device("cpu")
LR_net = LinearRegression(input_dim = d, output_dim = 1)
LR_net.to(device)
copied_model = copy.deepcopy(LR_net)
copied_model.to(device)
loss_func = nn.MSELoss()
lr0 = 1e-4
lr_schedule = 'constant'
b = 2
m = 32
max_batch_size = int(n/m)
beta_1 = torch.tensor(0.9)
beta_2 = torch.tensor(0.999)
sigma = torch.tensor(1e-8)


onr_gadam_loss_list = []
onr_gadam_gradnorm_list = []
onr_gadam_test_loss_list = []

#stage two: load training set and testing set
BS = BatchSampler(WeightedRandomSampler(torch.ones(n), replacement = False, num_samples = b*m), batch_size = b*m, drop_last = False)
LR_onr_train_loader = DataLoader(onr_train, batch_sampler = BS)
LR_onr_train_loader = DeviceDataLoader(LR_onr_train_loader, device)


#stage three: train and test 
batch_idx = torch.tensor(0)
h_0 = [torch.zeros_like(paras) for paras in list(LR_net.parameters())]
v_0 = [torch.zeros_like(paras) for paras in list(LR_net.parameters())]
for epoch in range(epoches*max_batch_size):
    LR_net.train()
    for x_data, y_target in LR_onr_train_loader:
        xt = []
        yt = []
        losst = torch.empty(2)
        xt = x_data.split(m, dim = 0)
        yt = y_target.split(m, dim = 0)
        if lr_schedule == 't-inverse':
            lr = lr0 * (1/int(1 + epoch + batch_idx/n))
        else:
            lr = lr0
        #calculate loss for first to derive the resampling probability
        for i, x in enumerate(xt):
            with torch.no_grad():
                output = LR_net(x)
                losst[i] = loss_func(output, yt[i].float().view(x.shape[0], -1)).item()
        prob = losst/torch.sum(losst)
        zero_grad(list(LR_net.parameters()))
        zero_grad(list(copied_model.parameters()))
        #construct the adam-based grafting gradient 
        output1 = LR_net(xt[0])
        loss1 = loss_func(output1, yt[0].float().view(xt[0].shape[0], -1))
        loss1.backward()
        output2 = copied_model(xt[1])
        loss2 = loss_func(output2, yt[1].float().view(xt[1].shape[0], -1))
        loss2.backward()
        for j, (p1, p2)  in enumerate(zip(list(LR_net.parameters()), list(copied_model.parameters()))):
            d_p1 = p1.grad.data
            d_p2 = p2.grad.data
            if weight_decay != 0:
                d_p1.add_(p1.data, alpha = weight_decay)
                d_p2.add_(p2.data, alpha = weight_decay)
            indices = torch.zeros_like(torch.clone(d_p1).detach())
            indices = indices.bernoulli_(p = prob[0]).to(torch.bool)
            d_p1.masked_fill_(~indices, 0)
            d_p2.masked_fill_(indices, 0)
            d_p1.mul_(1/b).mul_(1/prob[0])
            d_p2.mul_(1/b).mul_(1/prob[1])
            ggd_1 = torch.clone(d_p1).detach() + torch.clone(d_p2).detach()
            exp_avg = h_0[j]
            exp_avg_sq = v_0[j]
            exp_avg.mul_(beta_1).add_(ggd_1, alpha = 1 - beta_1)
            exp_avg_sq.mul_(beta_2).addcmul_(ggd_1, ggd_1.conj(), value = 1 - beta_2)
            bias_correction1 = 1 - torch.pow(beta_1, (batch_idx+1))
            bias_correction2 = 1 - torch.pow(beta_2, (batch_idx+1))
            step_size = lr / bias_correction1
            step_size_neg = step_size.neg()
            bias_correction2_sqrt = bias_correction2.sqrt()
            denom = (exp_avg_sq.sqrt() / (bias_correction2_sqrt * step_size_neg)).add_(sigma / step_size_neg)
            p1.data.addcdiv_(exp_avg, denom)
        copied_model = copy.deepcopy(LR_net)
        copied_model.to(device)
        batch_idx += 1
    if (epoch+1) % max_batch_size == 0:
        LR_net.eval()
        current_gradnorm = total_grad(LR_net, loss_func_rec, onr_train_loader, n)
        onr_gadam_gradnorm_list.append(current_gradnorm)
        with torch.no_grad():
            current_loss = total_loss(LR_net, loss_func_rec, onr_train_loader, n)
            onr_gadam_loss_list.append(current_loss)
            current_test_loss = test_loss(LR_net, loss_func_rec, onr_test_loader, n1)
            onr_gadam_test_loss_list.append(current_test_loss)
            current_iteration =  (epoch+1)/max_batch_size
        print('Iteration: {}  Loss: {}  Gradnorm:{}'.format(current_iteration, current_loss, current_gradnorm))

Iteration: 1.0  Loss: 0.004618581950467185  Gradnorm:0.04206166788935661
Iteration: 2.0  Loss: 0.0018463852276622394  Gradnorm:0.024229800328612328
Iteration: 3.0  Loss: 0.0009843065002711436  Gradnorm:0.00929498765617609
Iteration: 4.0  Loss: 0.0006025709861388215  Gradnorm:0.006281221751123667
Iteration: 5.0  Loss: 0.0004097160479914554  Gradnorm:0.0032170673366636038
Iteration: 6.0  Loss: 0.00032122556355667476  Gradnorm:0.0047879875637590885
Iteration: 7.0  Loss: 0.00026979031936614535  Gradnorm:0.007563370745629072
Iteration: 8.0  Loss: 0.00023727505252730784  Gradnorm:0.0035604117438197136
Iteration: 9.0  Loss: 0.0002218866672342778  Gradnorm:0.007257960736751556
Iteration: 10.0  Loss: 0.00021083068608269788  Gradnorm:0.0015693921595811844
Iteration: 11.0  Loss: 0.00020382682382167748  Gradnorm:0.0017238575965166092
Iteration: 12.0  Loss: 0.00019889793292817947  Gradnorm:0.0016991322627291083
Iteration: 13.0  Loss: 0.00019932198272872644  Gradnorm:0.002504534088075161
Iteration: 

In [10]:
#training stage: Linear regression with l^2 regularization using ggd-as 
#stage one: preparation, initialization and hyperparameter setting
device = torch.device("cpu")
LR_net = LinearRegression(input_dim = d, output_dim = 1)
LR_net.to(device)
copied_model = copy.deepcopy(LR_net)
copied_model.to(device)
loss_func = nn.MSELoss()
lr0 = 0.1
lr_schedule = 't-inverse'
b = 2
m = 32
max_batch_size = int(n/m)
onr_ggdas_loss_list = []
onr_ggdas_gradnorm_list = []
onr_ggdas_test_loss_list = []


#stage two: load training set and testing set
BS = BatchSampler(WeightedRandomSampler(torch.ones(n), replacement = False, num_samples = b*m), batch_size = b*m, drop_last = False)
LR_onr_train_loader = DataLoader(onr_train, batch_sampler = BS)
LR_onr_train_loader = DeviceDataLoader(LR_onr_train_loader, device)


#stage three: train and test
for epoch in range(epoches*max_batch_size):
    LR_net.train()
    for x_data, y_target in LR_onr_train_loader:
        xt = []
        yt = []
        losst = torch.empty(2)
        xt = x_data.split(m, dim = 0)
        yt = y_target.split(m, dim = 0)
        #calculate loss for first to derive the resampling probability
        for i, x in enumerate(xt):
            with torch.no_grad():
                output = LR_net(x)
                losst[i] = loss_func(output, yt[i].float().view(x.shape[0], -1)).item()
        prob = losst/torch.sum(losst)
        zero_grad(list(LR_net.parameters()))
        zero_grad(list(copied_model.parameters()))
        if lr_schedule == 't-inverse':
            lr = lr0 * (1/int(epoch/(5*max_batch_size)+1))
        else:
            lr = lr0
        #construct the grafting gradient
        output1 = LR_net(xt[0])
        output2 = copied_model(xt[1])
        loss1 = loss_func(output1, yt[0].float().view(xt[0].shape[0], -1))
        loss2 = loss_func(output2, yt[1].float().view(xt[1].shape[0], -1))
        loss1.backward()
        loss2.backward()
        for  p1, p2 in zip(list(LR_net.parameters()), list(copied_model.parameters())):
            d_p1 = p1.grad.data
            d_p2 = p2.grad.data
            if weight_decay != 0:
                d_p1.add_(p1.data, alpha = weight_decay)
                d_p2.add_(p2.data, alpha = weight_decay)
            indices = torch.zeros_like(torch.clone(d_p1).detach())
            indices = indices.bernoulli_(p = prob[0]).to(torch.bool)
            d_p1.masked_fill_(~indices, 0)
            d_p2.masked_fill_(indices, 0)
            d_p1.mul_(1/b).mul_(1/prob[0])
            d_p2.mul_(1/b).mul_(1/prob[1])
            p1.data.add_(torch.add(d_p1, d_p2), alpha = -lr)
        copied_model = copy.deepcopy(LR_net)
        copied_model.to(device)    
    if (epoch+1) % max_batch_size == 0:
        LR_net.eval()
        current_gradnorm = total_grad(LR_net, loss_func_rec, onr_train_loader, n)
        onr_ggdas_gradnorm_list.append(current_gradnorm)
        with torch.no_grad():
            current_loss = total_loss(LR_net, loss_func_rec, onr_train_loader, n)
            onr_ggdas_loss_list.append(current_loss)
            current_test_loss = test_loss(LR_net, loss_func_rec, onr_test_loader, n1)
            onr_ggdas_test_loss_list.append(current_test_loss)
        current_iteration =  int((epoch+1)/max_batch_size)
        print('Iteration: {}  Loss: {}  Gradnorm:{}'.format(current_iteration, current_loss, current_gradnorm)) 

Iteration: 1  Loss: 0.00033541151059457566  Gradnorm:0.04113851860165596
Iteration: 2  Loss: 0.0002160087850367104  Gradnorm:0.008576749823987484
Iteration: 3  Loss: 0.00020230722135124046  Gradnorm:0.0047256723046302795
Iteration: 4  Loss: 0.0002004726707521869  Gradnorm:0.010763927362859249
Iteration: 5  Loss: 0.00020953042562754396  Gradnorm:0.02246834523975849
Iteration: 6  Loss: 0.000191614780725975  Gradnorm:0.0015589187387377024
Iteration: 7  Loss: 0.00019081645920795165  Gradnorm:0.0032060574740171432
Iteration: 8  Loss: 0.00018906331568693262  Gradnorm:0.002086319262161851
Iteration: 9  Loss: 0.00019015422410676857  Gradnorm:0.006270367186516523
Iteration: 10  Loss: 0.00018820835627766979  Gradnorm:0.0005005186540074646
Iteration: 11  Loss: 0.00018853894912790618  Gradnorm:0.0017259467858821154
Iteration: 12  Loss: 0.0002089522404873125  Gradnorm:0.02331104874610901
Iteration: 13  Loss: 0.00019202047441258674  Gradnorm:0.010758776217699051
Iteration: 14  Loss: 0.00018813946736

In [5]:
#training stage: Linear regression with l^2 regularization using ggd-svrg
#stage one: preparation, initialization and hyperparameter setting
device = torch.device("cpu")
LR_net = LinearRegression(input_dim = d, output_dim = 1)
Snap_model = copy.deepcopy(LR_net)
LR_net.to(device)
Snap_model.to(device)
loss_func = nn.MSELoss()
lr0 = 0.1
lr_schedule = 'constant'
b = 2
m = 32
max_batch_size = int(n/m)
q = max_batch_size
onr_gsvrg_loss_list = []
onr_gsvrg_gradnorm_list = []
onr_gsvrg_test_loss_list = []


#stage two: load training set and testing set
BS = BatchSampler(WeightedRandomSampler(torch.ones(n), replacement = False, num_samples = b*m), batch_size = b*m, drop_last = False)
LR_onr_train_loader = DataLoader(onr_train, batch_sampler = BS)
LR_onr_train_loader = DeviceDataLoader(LR_onr_train_loader, device)


#stage three: train and test 
fg = full_grad(Snap_model, loss_func_rec, onr_train_loader, n)
batch_idx = 0
for epoch in range(epoches*max_batch_size):
    LR_net.train()
    for x_data, y_target in LR_onr_train_loader:
        g0 = []
        g1 = []
        gf_r0 = []
        gf_r1 = []
        ggd = []
        xt = []
        yt = []
        losst = torch.empty(2)
        xt = x_data.split(m, dim = 0)
        yt = y_target.split(m, dim = 0)
        if lr_schedule == 't-inverse':
            lr = lr0 * (1/int(1 + epoch + batch_idx/n))
        else:
            lr = lr0
        #construct the svrg-based grafting gradient
        #part one: prepare for g_mb(bar{x})
        for i, x in enumerate(xt):
            output = Snap_model(x)
            loss_snap = loss_func(output, yt[i].float().view(x.shape[0], -1))
            loss_snap.backward()
            for j, p in enumerate(list(Snap_model.parameters())):
                d_p = p.grad.data
                if weight_decay != 0:
                    d_p.add_(p.data, alpha = weight_decay)
                if i == 0:
                    gf_r0.append(torch.clone(d_p).detach())
                else:
                    gf_r1.append(torch.clone(d_p).detach())
            zero_grad(list(Snap_model.parameters()))
        norm_2 = torch.zeros(2)
        #deriving the sampling probability and preparing for g_mb(x^k_s)
        for i, x in enumerate(xt):
            output = LR_net(x)
            loss = loss_func(output, yt[i].float().view(x.shape[0], -1))
            loss.backward()
            if i == 0:
                for z, p in zip(gf_r0, list(LR_net.parameters())):
                    d_p = p.grad.data
                    if weight_decay != 0:
                        d_p.add_(p.data, alpha = weight_decay)
                    g0.append(torch.clone(d_p).detach())
                    
                    norm_2[i] = norm_2[i] + torch.sum(torch.square(torch.add(z, torch.clone(d_p).detach(), alpha = -1)))
            else:
                for z, p in zip(gf_r1, list(LR_net.parameters())):
                    d_p = p.grad.data
                    if weight_decay != 0:
                        d_p.add_(p.data, alpha = weight_decay)
                    g1.append(torch.clone(d_p).detach())
                    
                    norm_2[i] = norm_2[i] + torch.sum(torch.square(torch.add(z, torch.clone(d_p).detach(), alpha = -1)))
            zero_grad(list(LR_net.parameters()))
        if torch.min(norm_2) == 0:
            norm_2 = torch.ones(2)
        prob = torch.sqrt(norm_2)/torch.sum(torch.sqrt(norm_2))
        #constructing the grafting gradient \tilde{g}^k_mb
        for qr, qo, pr, po, fg_p in zip(gf_r0, g0, gf_r1, g1, fg):
            indices = torch.zeros_like(qr)
            indices = indices.bernoulli_(p = prob[0]).to(torch.bool)
            qr.masked_fill_(~indices, 0)
            qo.masked_fill_(~indices, 0)
            pr.masked_fill_(indices, 0)
            po.masked_fill_(indices, 0)
            ggd.append(torch.add(po, qo) - torch.add(pr, qr) + fg_p)
        
        #update!
        for g, p in zip(ggd, list(LR_net.parameters())):
            p.data = torch.add(p.data, g, alpha = -lr)
        batch_idx += 1
        #Break the loop when iteration number equal update frequency
    if  batch_idx  % q == 0:
        Snap_model = copy.deepcopy(LR_net)
        fg = full_grad(Snap_model, loss_func_rec, onr_train_loader, n)
    if batch_idx % max_batch_size == 0:
        LR_net.eval()
        current_gradnorm = total_grad(LR_net, loss_func_rec, onr_train_loader, n)
        onr_gsvrg_gradnorm_list.append(current_gradnorm)
        with torch.no_grad():
            current_loss = total_loss(LR_net, loss_func_rec, onr_train_loader, n)
            onr_gsvrg_loss_list.append(current_loss)
            current_test_loss = test_loss(LR_net, loss_func_rec, onr_test_loader, n1)
            onr_gsvrg_test_loss_list.append(current_test_loss)
            current_iteration = batch_idx / max_batch_size
            print('Iteration: {}  Loss: {}  Gradnorm:{}'.format(current_iteration, current_loss, current_gradnorm))       

Iteration: 1.0  Loss: 0.009064189947574236  Gradnorm:0.5098594427108765
Iteration: 2.0  Loss: 0.00036832878788093016  Gradnorm:0.02865799330174923
Iteration: 3.0  Loss: 0.00022537484518383725  Gradnorm:0.0045554074458777905
Iteration: 4.0  Loss: 0.00020851223489782629  Gradnorm:0.0018601295305415988
Iteration: 5.0  Loss: 0.0002004540025431445  Gradnorm:0.0010745780309662223
Iteration: 6.0  Loss: 0.00019547760221727184  Gradnorm:0.0007609976455569267
Iteration: 7.0  Loss: 0.0001922941586522768  Gradnorm:0.0014818457420915365
Iteration: 8.0  Loss: 0.00019003919024920777  Gradnorm:0.0003607641556300223
Iteration: 9.0  Loss: 0.00018852756308908815  Gradnorm:0.00044145958963781595
Iteration: 10.0  Loss: 0.00018744819696118136  Gradnorm:0.0004522191593423486
Iteration: 11.0  Loss: 0.00018665942597042475  Gradnorm:0.00020035200577694923
Iteration: 12.0  Loss: 0.00018608549566613225  Gradnorm:0.000460511859273538
Iteration: 13.0  Loss: 0.0001856443662347336  Gradnorm:0.00018309729057364166
Ite